In [6]:
import keras
import datetime
import tensorflow                       as tf
from tensorflow.keras.callbacks         import TensorBoard
from tensorflow.keras.layers            import Input, Lambda,UpSampling2D, Conv2D,Dropout,MaxPooling2D,Conv2DTranspose,concatenate,BatchNormalization, Activation
from tensorflow.keras.models            import Model
from tensorflow.keras.optimizers        import Adam,RMSprop
from keras.utils                        import plot_model
from tensorflow.keras                   import layers, models
from tensorflow.keras.losses            import mae
import sys
import os
import numpy as np
import math
import random, time
from pathlib                        import Path
from PIL                            import Image

import skimage                      as ski
from   skimage.filters              import threshold_otsu
from   skimage                      import io, color
from   skimage.color                import rgb2gray
from   skimage                      import filters
import cv2                          as cv
import matplotlib.pyplot            as plt 
import gc
import glob
from skimage                        import img_as_ubyte
from skimage                        import io
import shutil
tf.keras.backend.clear_session()

#### 1. Load main directories

In [7]:
general_path_data = '/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5'
#general_path_data ='/home/guiomar/Desktop/CODES/Generation_Data/Proyecto_UNI'

subdirectories = ['/Geometry','/Magnitude','/Pression','/U001','/U002']

geo_path =  general_path_data + subdirectories[0]
v_path   =  general_path_data + subdirectories[1]
p_path   =  general_path_data + subdirectories[2]
vx_path  =  general_path_data + subdirectories[3]
vy_path  =  general_path_data + subdirectories[4]

#### 2. Methods for processing data

In [8]:
neWidth = 256
### method for reading image    
def get_img(img_name):
    return ski.io.imread(img_name)

###   method for resizing image
###   by default  64 x 256          another posibility 32 x 128
def resizing_img(x,new_width=neWidth):
   
    height=x.shape[0]
    width=x.shape[1]
    ratio = height / width
    new_height = int(new_width * ratio)
    y =cv.resize(x,(new_width,new_height))
    return y

### method for turning to a grey or binary image 
def processing(image,option):
        x = get_img(image)  
        x = rgb2gray(x)               # It returns a grayscale image with floating point values in the range from 0 to 1
        y = resizing_img(x) # Reshape image 
    
        # Binary option otherwise gray
        if (option):
            y=ski.util.img_as_ubyte(y)  # Convert it back to the original data type and the data range back 0 to 255. 
                                        # It is often better to use image values represented by floating point values 
            best_value_threshold=np.round(filters.threshold_otsu(y)) #  Otsu’s method calculates an “optimal” threshold

            _,y= cv.threshold(y, best_value_threshold, 255, cv.THRESH_BINARY)
            y=y/255.
        y=y[:, :, np.newaxis]
        return y

def create_file(path,name='Images_masked'):
     newpath = path +'/' +name
     os.makedirs(newpath, exist_ok=True)
     return newpath

###  Method to create  npy object and save in a disk
def create_npy(path_origin,name_sub_file,option,n_sample = 1000):
    images = sorted([os.path.join(path_origin,fname) for fname in os.listdir(path_origin) if fname.endswith(".png")])
    current_path = os.getcwd() 
    if (n_sample <= 20000):

        I = images[1:n_sample+1]
        newpath1=create_file(current_path)
        new_root = newpath1 +'/' +name_sub_file
        os.makedirs(new_root, exist_ok=True)
   
        for img in I:
            name_img = os.path.splitext(os.path.basename(img))[0]
            save_in=new_root+'/'+ name_img+'.npy'
            print(save_in)
            x=processing(img,option)
            np.save(save_in,x)

### Method to apply mask, the mask is done by multiplication of two arrays
def mask(path1,path2,save_in,new_npy):
    
    path_1 = Path(path1)
    file_npy_1=sorted(list(path_1.glob('*.npy')))
    
    path_2 = Path(path2)
    file_npy_2=sorted(list(path_2.glob('*.npy')))
    n=1

    for f1, f2 in zip(file_npy_1,file_npy_2):
            new_save_in=save_in+ '/'+new_npy+f'_{n:05d}.npy'
            print(new_save_in)
            array_1 = np.load(f1)
            array_2 = np.load(f2)
            array_masked=array_1*array_2
            np.save(new_save_in,array_masked)
            print('*********',new_save_in)
            n+=1

def create_file_masked(file1,file2,name_npy,name):
    path1 =  os.getcwd() 
    path2= create_file(path1,name)
    mask(file1,file2,path2,name_npy)                         

In [9]:
### Method for divided data in train,test and validation
def split_data(file,n_train,n_valid,n_test):
     n_sample=len(file)
     if (round(n_train + n_valid + n_test,2) == 1.0):
        n_train= math.floor(n_sample*n_train)
        n_valid = math.floor(n_sample*n_valid) + n_train
        n_test = math.floor(n_sample*n_test) + n_valid
        (train,valid,test) = (file[0:n_train], file[n_train:n_valid],file[n_valid:n_test]) 
     return train,valid,test

def create_files(ntrain,ntest,nval):
    
    name_files=['/G_Masked','/P_Masked','/V_Masked','/Vx_Masked','/Vy_Masked']
    current_path = os.getcwd()
    paths=[current_path+name_files[0],current_path+name_files[1],current_path+name_files[2],current_path+name_files[3],current_path+name_files[4]]
    for i, path in enumerate(paths):

        file = Path(path)
        arrays= sorted(file.glob('*.npy')) 
        train,valid,test =  split_data(arrays,ntrain,ntest,nval)

        path_train = paths[i]+'/train'
        path_test =  paths[i]+'/test'
        path_valid = paths[i]+'/valid'

        os.makedirs(path_train, exist_ok=True)
        os.makedirs(path_test, exist_ok=True)
        os.makedirs(path_valid, exist_ok=True)

        for array in train:
            p =  path_train + '/' + array.name
            shutil.move(array,p)

        for array in valid:
            p =  path_valid + '/' + array.name
            shutil.move(array,p)
        
        for array in test:
            p =  path_test+ '/' + array.name
            shutil.move(array,p)
        

### <font color="red">2. Executing</font>

In [10]:
sample=1000

create_npy(geo_path,'Geo',True,n_sample = sample)
create_npy(v_path ,'V',False,n_sample = sample)
create_npy(p_path,'P',False,n_sample = sample)
create_npy(vx_path ,'Vx',False,n_sample = sample)
create_npy(vy_path ,'Vy',False,n_sample = sample)


/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00001.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00002.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00003.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00004.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00005.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00006.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00007.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00008.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00009.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00010.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00011.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00012.npy
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo/geo_00013.npy
/home/ppgi/T

In [11]:
path1='/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Geo'
path2='/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/P'
path3='/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/V'
path4='/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Vx'
path5='/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/Vy'

list_paths = [path1,path2,path3,path4,path5]
labels = [('p','P_Masked'),('v','V_Masked'),('vx','Vx_Masked'),('vy','Vy_Masked')]

for k in range(len(labels)):
    create_file_masked(list_paths[0],list_paths[k+1],labels[k][0],labels[k][1])

/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00001.npy
********* /home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00001.npy
/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00002.npy
********* /home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00002.npy
/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00003.npy
********* /home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00003.npy
/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00004.npy
********* /home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00004.npy
/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00005.npy
********* /home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00005.npy
/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00006.npy
********* /home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00006.npy
/home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00007.npy
********* /home/ppgi/Trabajo/predicting-flow-patterns/P_Masked/p_00007.npy
/home/ppgi/Trabajo/p

In [12]:
ntrain = 0.8
ntest  = 0.1
nval   = 0.1

create_files(ntrain,ntest,nval)